In [1]:
from typing_extensions import TypedDict
from langgraph.graph.state import StateGraph, START

In [3]:
from tracemalloc import start


class State(TypedDict):
    foo:str

def sub_graph_node_1(state:State):
    return {"foo":'hi'+state['foo']}

In [4]:
builder = StateGraph(State)
builder.add_node("node_1", sub_graph_node_1)
builder.add_edge(START, "node_1")
# builder.add_edge("node_1", END)

graph = builder.compile()

graph.invoke({"foo":"world"})

{'foo': 'hiworld'}

# Shared state schemas

In [5]:
class SubgraphState(TypedDict):
    foo:str
    bar:str

def subgraph_node_1(state: SubgraphState):
    return {"bar": "bar"}

def subgraph_node_2(state: SubgraphState):
    # note that this node is using a state key ('bar') that is only available in the subgraph
    # and is sending update on the shared state key ('foo')
    return {"foo": state["foo"] + state["bar"]}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph = subgraph_builder.compile()

In [7]:
subgraph.invoke({"foo":"foo"})

{'foo': 'foobar', 'bar': 'bar'}

In [ ]:
# Define parent graph
class ParentState(TypedDict):
    foo: str
    big:str

def node_1(state: ParentState):
    return {"foo": "hi! " + state["foo"]}

builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", subgraph)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
graph = builder.compile()

for chunk in graph.stream({"foo": "foo1"}):
    print(chunk)

# In this example, they share the same state key 'foo'
# but the state key 'big' is only available in the parent graph
# and is not available in the subgraph
# so when we invoke the graph, the state key 'big' is not available in the subgraph
# and the subgraph will not be able to update the state key 'big'
# so the state key 'big' is not available in the subgraph
# and the subgraph will not be able to update the state key 'big'

{'node_1': {'foo': 'hi! foo1'}}
{'node_2': {'foo': 'hi! foo1bar'}}


# Different state schemas

In [ ]:
# Define subgraph
class SubgraphState(TypedDict):
    # note that none of these keys are shared with the parent graph state
    bar: str
    baz: str

def subgraph_node_1(state: SubgraphState):
    return {"baz": "baz"}

def subgraph_node_2(state: SubgraphState):
    return {"bar": state["bar"] + state["baz"]}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph = subgraph_builder.compile()

# Define parent graph
class ParentState(TypedDict):
    foo: str

def node_1(state: ParentState):
    return {"foo": "hi! " + state["foo"]}

def node_2(state: ParentState):
    response = subgraph.invoke({"bar": state["foo"]})  
    return {"foo": response["bar"]}  


builder = StateGraph(ParentState)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
graph = builder.compile()

for chunk in graph.stream({"foo": "foo"}, subgraphs=True):
    print(chunk)

# in this func, call the subgraph in main graph, and the state of the subgraph is not shared with the main graph
# just use the response of the subgraph as the input of the main graph

((), {'node_1': {'foo': 'hi! foo'}})
(('node_2:1073ff85-ad71-4ea1-9665-97d7cb5b37cd',), {'subgraph_node_1': {'baz': 'baz'}})
(('node_2:1073ff85-ad71-4ea1-9665-97d7cb5b37cd',), {'subgraph_node_2': {'bar': 'hi! foobaz'}})
((), {'node_2': {'foo': 'hi! foobaz'}})
